In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from src.features import build_features

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [3]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO

In [4]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [5]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [35]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados = pd.concat(poblaciones_estratos_calculados)
poblaciones_estratos_calculados = poblaciones_estratos_calculados.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

# Guarda resultados
poblaciones_estratos_calculados.to_excel("../data/interim/poblacion_INE_estratos.xlsx", index=False)

---

In [36]:
RUTA_PLANILLA_INCIDENCIAS = "../data/processed/proyeccion_problemas_de_salud.xlsx"
COLUMNAS_A_UTILIZAR = ["Diagnostico", "Incidencia (Cada 100.000)", "Edad Incidencia"]

# Lee planilla de incidencias
incidencias = pd.read_excel(
    RUTA_PLANILLA_INCIDENCIAS, sheet_name="incidencias_prevalencias", usecols=COLUMNAS_A_UTILIZAR
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Incidencia (Cada 100.000)"] / 100000

In [37]:
incidencias.merge(poblaciones_estratos_calculados, how="left", on="Edad Incidencia")

,Diagnostico,Incidencia (Cada 100.000),Edad Incidencia,rate_incidencia,Estrato,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021,...,Poblacion 2026,Poblacion 2027,Poblacion 2028,Poblacion 2029,Poblacion 2030,Poblacion 2031,Poblacion 2032,Poblacion 2033,Poblacion 2034,Poblacion 2035
0,C33X - Tumor maligno de la tráquea,0.26,todos,0.000003,Pais,18419192.0,18751405.0,19107216.0,19458310.0,19678363.0,...,20322807.0,20433791.0,20539666.0,20640230.0,20735289.0,20826397.0,20912278.0,20992831.0,21067996.0,21137769.0
1,C33X - Tumor maligno de la tráquea,0.26,todos,0.000003,Tarapacá,342129.0,354940.0,368906.0,382773.0,391165.0,...,415447.0,419920.0,424315.0,428636.0,432868.0,437075.0,441216.0,445290.0,449291.0,453219.0
2,C33X - Tumor maligno de la tráquea,0.26,todos,0.000003,Antofagasta,623851.0,645022.0,668563.0,691854.0,703746.0,...,725843.0,729405.0,732792.0,736010.0,739057.0,742188.0,744984.0,747617.0,750069.0,752337.0
3,C33X - Tumor maligno de la tráquea,0.26,todos,0.000003,Atacama,304580.0,307835.0,311307.0,314709.0,316737.0,...,321644.0,322351.0,322990.0,323566.0,324061.0,324510.0,324890.0,325209.0,325448.0,325620.0
4,C33X - Tumor maligno de la tráquea,0.26,todos,0.000003,Coquimbo,793049.0,807213.0,821726.0,836096.0,848079.0,...,899128.0,908851.0,918417.0,927839.0,937116.0,946324.0,955364.0,964216.0,972858.0,981301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,Q676 - Tórax excavado,143.00,recien_nacidos_vivos,0.001430,Ñuble,5518.0,5273.0,5237.0,5237.0,5233.0,...,5068.0,5022.0,4969.0,4915.0,4857.0,4836.0,4786.0,4741.0,4688.0,4641.0
637,Q676 - Tórax excavado,143.00,recien_nacidos_vivos,0.001430,SSMO,16070.0,16006.0,16346.0,16916.0,17242.0,...,16700.0,16474.0,16221.0,15958.0,15688.0,15532.0,15307.0,15078.0,14859.0,14652.0
638,T820 - Complicación mecánica de prótesis de vá...,3050.00,Pacientes al año con prótesis,0.030500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,T821 - Complicación mecánica de dispositivo el...,3050.00,Pacientes al año con prótesis,0.030500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
